## Homework 9

In this homework, we'll deploy the dino or dragon model we trained in the previous homework.

Download the model from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5)

### Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?

- 21 Mb

- 43 Mb

- 80 Mb

- 164 Mb

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras_image_helper import create_preprocessor

In [5]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

2022-11-27 08:12:39.749023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('dino_dragon.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/mp/981dprsj49dbyd4clvj123sm0000gn/T/tmpv1ou4g4x/assets


INFO:tensorflow:Assets written to: /var/folders/mp/981dprsj49dbyd4clvj123sm0000gn/T/tmpv1ou4g4x/assets
2022-11-27 08:15:06.458564: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-27 08:15:06.458587: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-27 08:15:06.458725: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/mp/981dprsj49dbyd4clvj123sm0000gn/T/tmpv1ou4g4x
2022-11-27 08:15:06.460337: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-27 08:15:06.460351: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/mp/981dprsj49dbyd4clvj123sm0000gn/T/tmpv1ou4g4x
2022-11-27 08:15:06.465815: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-27 08:15:06.548193: I tensorflow/cc/saved_model/loader.cc:213] Running initialization

In [11]:
!ls -lh

total 262968
-rw-r--r--   1 tatyjanaankudo  staff   7.8K Nov 27 08:16 HomeWork_9.ipynb
drwxr-xr-x  11 tatyjanaankudo  staff   352B Nov 26 22:15 Theory
drwxr-xr-x   3 tatyjanaankudo  staff    96B Nov 26 18:56 __pycache__
-rw-r--r--   1 tatyjanaankudo  staff    43M Nov 27 08:15 dino_dragon.tflite
-rw-r--r--@  1 tatyjanaankudo  staff    86M Nov 27 07:34 dino_dragon_10_0.899.h5
-rw-r--r--   1 tatyjanaankudo  staff   278B Nov 27 07:48 test.py


`Answer - 43M`

### Question 2

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

- 3

- 7

- 13

- 24

In [12]:
import tensorflow.lite as tflite

In [14]:
interpreter = tflite.Interpreter(model_path='dino_dragon.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [15]:
output_index

13

`Answer - 13`

Preparing the image

You'll need some code for downloading and resizing images. You can use this code:

In [16]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

For that, you'll need to have pillow installed:

Let's download and resize this [image](https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg):

Based on the previous homework, what should be the target size for the image?

In [18]:
url_img = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'

In [32]:
IMG_SIZE = (150, 150)
img = download_image(url_img)
img = prepare_image(img, IMG_SIZE)

/var/folders/mp/981dprsj49dbyd4clvj123sm0000gn/T/ipykernel_673/2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


### Question 3

Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?
After the pre-processing, what's the value in the first pixel, the R channel?

- 0.3353411

- 0.5529412

- 0.7458824

- 0.9654902

In [29]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [39]:
x = np.array(img, dtype='float32')
X = np.array([x])/255

X = X.astype(np.float32)

In [40]:
X[0][0,0,0]

0.5529412

`Answer 0.5529412`

### Question 4

Now let's apply this model to this image. What's the output of the model?

- 0.17049132

- 0.39009996

- 0.60146114

- 0.82448614

In [42]:
X.shape

(1, 150, 150, 3)

In [43]:
preds = model.predict(X)

1/1 [==============================] - 0s 139ms/step


In [44]:
preds

array([[0.82448584]], dtype=float32)

`Answer 0.82448584`

#### Prepare the lambda code

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with ipython or Jupyter Notebook by importing the file and invoking the function from this file.

#### Docker

For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

```
FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
```

And pushed it to svizor42/zoomcamp-dino-dragon-lambda:v2.

A few notes:

The image already contains a model and it's not the same model as the one we used for questions 1-4.
The version of Python is 3.9, so you need to use the right wheel for TF-Lite. For Tensorflow 2.7.0, [it's](https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.7.0-cp39-cp39-linux_x86_64.whl)

### Question 5

Download the base image svizor42/zoomcamp-dino-dragon-lambda:v2. You can easily make it by using docker pull command.

So what's the size of this base image?

- 139 Mb

- 329 Mb

- 639 Mb

- 929 Mb

You can get this information when running docker images - it'll be in the "SIZE" column.

Build container and test:

```
$ docker build -t dino_dragon 

$ docker run -it --rm -p 8080:8080 dino_dragon:latest

$ python request.py

```

To get size of the image:

```
$ docker inspect -f "{{ .Size }}" svizor42/zoomcamp-dino-dragon-lambda:v2
```


docker images dino_dragon

`answer: 639M`

### Question 6

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is dino-vs-dragon-v2.tflite and it's in the current workdir in the image (see the Dockerfile above for the reference).

Now run the container locally.

Score this [image](https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg)

What's the output from the model?

- 0.12

- 0.32

- 0.52

- 0.72

```
$ docker build -t dino_dragon .

$ run : docker run -it --rm -p 8080:8080 dino_dragon

$ run : python request.py
```


`Answer 0.32`

Publishing it to AWS

Now you can deploy your model to AWS!

Publish your image to ECR
Create a lambda function in AWS, use the ECR image
Give it more RAM and increase the timeout
Test it
Expose the lambda function using API Gateway
This is optional and not graded.

Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

docker build -t zoomcamp-dino-dragon-lambda .
docker tag zoomcamp-dino-dragon-lambda:latest svizor42/zoomcamp-dino-dragon-lambda:v2
docker push svizor42/zoomcamp-dino-dragon-lambda:v2
Submit the results

Submit your results here: https://forms.gle/Pnx563ELg9jgjxHX6
You can submit your solution multiple times. In this case, only the last submission will be used
If your answer doesn't match options exactly, select the closest one